## Authorship attribution with Burrows' Delta

In this exercise we'll put together some of the things we've learned about textual analysis to see whether we can distinguish two authors of mystery and suspense around the turn of the 20th century. 

On one hand, we have a set of short stories by [E. Nesbit](https://en.wikipedia.org/wiki/E._Nesbit). She's known mostly as a children's writer, but she also wrote horror and mystery stories, and I've selected some of those (from her collection *Grim Tales*). For comparison, I've chosen a selection of short stories by Arthur Conan Doyle. 

### Preliminaries

#### Create a local copy of the corpus
Since we're running this in Google Colab, we have to download the corpus separately from the git repo.

In [ ]:
# Colab only
!rm -rf clas-3801 corpus
!git clone https://github.com/cwf2/clas-3801-fa23
!mv clas-3801-fa23/Week_05/corpus .
!rm -rf clas-3801-fa23

#### import statements

In [ ]:
import os
import requests
import pandas as pd
import spacy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

#### Load the language model

We're going to be using SpaCy for tokenization and lemmatization in this case, rather than the builtin Python string manipulation methods. The first step is to load an English language model.

In [ ]:
nlp = spacy.load('en_core_web_sm')

### Step 1: Read the files

We're going to read the files from a local directory, parse them, and organize the tokenized text into a single table with Pandas.

#### First, scan the directory for available texts

In [ ]:
# read all the filenames from the corpus directory
dir_texts = 'corpus'
files = [f for f in os.listdir(dir_texts) if not f.startswith('.')]

#### Loop over all the files and parse them

In this `for` loop we:
- read the text of the file into a string
- process the string with SpaCy to produce a Doc object
- extract just the tokens from the doc
- store them along with title and author in a dictionary
- append the dictionary to a growing list of dictionaries

In [ ]:
# start with an empty list
corpus = []

# loop over files
for i, filename in enumerate(files):
    # extract the author and title from the filename
    author, title = filename[:-4].split('_', 1)

    # add back the directory name to get a path to the file
    path = os.path.join('corpus', filename)

    # read the file
    print(f'[{i+1}/{len(files)}] {path}', end='...')
    
    with open(path) as f:
        fulltext = f.read()

        # parse with SpaCy pipeline
        doc = nlp(fulltext)
        # extract just the tokens
        tokens = [tok for tok in doc]
        print(len(doc), 'tokens')

        # add a new record to the list
        corpus.append(dict(
            author = author,
            title = title,
            token = tokens,
        ))

#### Convert our data to a Pandas DataFrame

Here we turn our list of dictionaries into a DataFrame to make it easer to see.

In [ ]:
# convert list of dictionaries to a Data Frame
corpus = pd.DataFrame(corpus)
display(corpus)

#### Break out the tokens into separate rows

Right now, our table has one row per work, and the `token` column contains a list for each row. Next we're going to use the [`explode()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) method to break out the tokens into separate rows. For example, each token in "From the Dead" will now have its own row, with the `author` and `title` values copied from the original row.

In [ ]:
# break out the `token` column
tokens = pd.DataFrame(corpus).explode('token', ignore_index=True)
display(tokens)

### Step 2: Feature Extraction

Right now, the `token` column is displaying in Jupyter as if it just held the text of the tokens, but really this column contains SpaCy Token objects, each containing a bunch of complex linguistic data. I want to extract three attributes from each token that I can use as features:
- `text`: a string representing how it looked in the original story
- `lemma_`: a string representing its **lemma** or dictionary headword
- `pos_`: a string representing its **part of speech**

Note the underscores following `lemma_` and `pos_`. If you don't add the underscore, then SpaCy will give you a numeric code for these features instead of a string.

I'm assigning the new features to columns called `text`, `lemma` and `pos`.

In [ ]:
# add `text` column
tokens['text'] = [tok.text for tok in tokens['token']]

# add `lemma` column
tokens['lemma'] = [tok.lemma_ for tok in tokens['token']]

# add `pos` column
tokens['pos'] = [tok.pos_ for tok in tokens['token']]

display(tokens)

#### Tidy up the data a little

The table now has one row for every token in the corpus, with columns for the text, lemma, and part of speech. But it's clear that some of the things SpaCy considers tokens aren't words, like punctuation marks, numerals, and some kinds of whitespace. 

Here I'm going to omit rows according to certain criteria.

In [ ]:
# keep only rows where part of speech is not punctuation
tokens = tokens.loc[tokens['pos'] != 'PUNCT']

# keep only rows where part of speech is not space
tokens = tokens.loc[tokens['pos'] != 'SPACE']

# keep only rows where part of speech is not proper noun (i.e. personal names)
tokens = tokens.loc[tokens['pos'] != 'PROPN']

# keep only rows where the token text has at least one letter in it
tokens = tokens.loc[tokens['text'].str.contains(r'[A-Za-z]')]

display(tokens)

### Step 3: Feature vectors for each work

Now that we have all the features broken out, we can calculate some statistics that will allow us to characterize each work as a bundle of features.

#### Part of speech features

Let's start by looking at how each text uses different parts of speech. We'll use the `crosstab()` function to get a count for each unique value in `pos`, for each unique combination of `author` and `title`:

In [ ]:
pos_count = pd.crosstab([tokens['author'], tokens['title']], tokens['pos'])
display(pos_count)

#### Normalization

This is already interesting, but of course, some of the texts are longer than others, so the raw counts might not be the most helpful way to look at part of speech use. Instead, we'll calculate how many times each part of speech is used per 1000 tokens.

By passing the `normalize` argument to `crosstab()` we can ask Pandas to divide each value by the total for its row (i.e., normalize by *index*). Since each row is one work, this tells us the fraction of all tokens in the work represented by each column. Then we multiply by 1000 just to make the numbers easier to read. We've now converted counts to frequencies.

In [ ]:
pos_freq = pd.crosstab([tokens['author'], tokens['title']], tokens['pos'], normalize="index")*1000
display(pos_freq)

#### Plotting

Let's create a simple plot. We'll use the x-axis to represent the frequency of one part of speech, and the y-axis to represent a different frequency. Each text will be located somewhere in the cartesian space defined by these two features.

I'm using Pyplot to create my graph. First I have to create a [Figure and Axes](https://matplotlib.org/stable/users/explain/axes/axes_intro.html), and then I can use the Axes to add elements to the graph.

In [ ]:
# define the features I want to look at
feat_x = 'NOUN'
feat_y = 'VERB'

# extract the list of author names from the table
#   - this is for labelling the points
authors = pos_freq.index.get_level_values(0)

# instantiate a new figure and axes
fig, ax = plt.subplots()

# add a new series to the plot for each author
for label, group in pos_freq.groupby(authors):
    ax.plot(group[feat_x], group[feat_y], marker='o', ls='', label=label)

# label the plot axes
ax.set_xlabel(feat_x)
ax.set_ylabel(feat_y)

# create a legend
ax.legend()

# display results
plt.show()

##### Try it out:

🤔 Try changing the part of speech values above and re-running the plot. Are some parts of speech better than others and differentiating the two authors?

### Lemma-based features

Let's try working with a larger feature set. This time, we'll use lemmas instead of part of speech tags. There are only 14 parts of speech in our table, but there are a lot of lemmas in this corpus.

Let's calculate the total number of occurrences for each lemma. This time, I'll use the `groupby()` method to split the table by lemmas. Then I'll **aggregate** the groups using the `agg()` method. This turns each group into a single value, based on whatever **aggregation function** I choose. If I wanted, I could aggregate different columns using different methods. 

For now, I'm just going to aggregate the `token` column and I'm going to use a simple count. The result will be a table with one row per unique lemma, and a column containing the count of the number of tokens that have that lemma.

In [ ]:
# count tokens to produce a new table
token_count = tokens.groupby('lemma').agg(count=('token', 'count'))

# sort by count, decreasing order
token_count = token_count.sort_values('count', ascending=False)

display(token_count)

So there are 6345 unique lemmas in this corpus. How many are *hapax legomena*, occurring only once? I can use the fact that the `sum()` function counts `True` as 1 and `False` as 0:

In [ ]:
sum(token_count['count'] == 1)

Wow! So out of 6345 lemmas, more than a third are hapax legomena. This graph has a long tail! Does it follow Zipf's law?

In [ ]:
token_count.plot(loglog=True)
plt.show()

#### Feature selection

For authorship attribution, we want to work with just the function words. Generally this means working with the *n* **most frequent words** (MFW). The value of *n* is something that we might need to tune through trial and error. Let's start with the top 100.

In [ ]:
token_count.iloc[:30]

We'll take the row names from the top *n* lines and use that as our "keep" list.

In [ ]:
mfw = token_count.iloc[:30].index.values
print(mfw)

I'll create a list of boolean values marking which rows in the token table match words in my feature set. I can use this as a mask to select just those rows in future operations.

In [ ]:
selected = tokens.lemma.isin(mfw)

#### Calculate the feature vectors

Here we'll use `crosstab()` again, just as with the part of speech features. The only difference is I'm masking the token table so that I'm only working on rows that match my feature set.

In [ ]:
lemma_count = pd.crosstab([tokens.loc[selected, 'author'],tokens.loc[selected, 'title']], tokens.loc[selected, 'lemma'])
display(lemma_count)

#### Normalize

As before, we want to switch from raw counts per text to frequencies, so that shorter and longer texts become more comparable. The only problem here is that we've left out most of the tokens, so we can't just divide by the row totals anymore. Instead, we'll have to go back and calculate how many tokens are in each text in a separate table.

In [ ]:
# a table of lemma counts
n_lemmas = tokens.groupby('title').agg(
    n=('lemma', 'count'),
    
)

# normalize feature vectors by lemma counts per text
lemma_freq = lemma_count.div(n_lemmas.n, axis=0) * 1000
display(lemma_freq)

### Part 4: Zeta scores

In [ ]:
lemma_z = lemma_freq.sub(lemma_freq.mean(), axis=1).div(lemma_freq.std(), axis=1)
display(lemma_z)

In [ ]:
feat_x = 'and'
feat_y = 'but'

fig, ax = plt.subplots()
for label, group in lemma_z.groupby(authors):
    ax.plot(group[feat_x], group[feat_y], marker='o', ls='', label=label)
ax.set_xlabel(feat_x)
ax.set_ylabel(feat_y)
ax.legend()

In [ ]:
pca_model = PCA(n_components=3)

In [ ]:
pca = pd.DataFrame(
    pca_model.fit_transform(lemma_z),
    index = lemma_z.index,
    columns = ['PC1', 'PC2', 'PC3'],
)
display(pca)

In [ ]:
feat_x = 'PC1'
feat_y = 'PC2'

fig, ax = plt.subplots()
for label, group in pca.groupby(authors):
    ax.plot(group[feat_x], group[feat_y], marker='o', ls='', label=label)
ax.set_xlabel(feat_x)
ax.set_ylabel(feat_y)
ax.legend()